In [1]:
import os

In [2]:
%pwd

'/Users/sumanp/Desktop/end_to_end_using_mlops/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/sumanp/Desktop/end_to_end_using_mlops'

In [5]:
# defining the entiry class for the data ingestion config

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
# defifning the configuration manager

from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml , create_directories

# configuration manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

logs/running_logs.log


In [7]:
# component class for data ingestion

import os
import urllib.request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size
import requests
import time

#components 

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = urllib.request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded !")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")


    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  


In [8]:
#pipeline

try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-09 18:34:24,071: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-09 18:34:24,073: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-09 18:34:24,075: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-02-09 18:34:24,076: INFO: common: created directory at: artifacts]
[2024-02-09 18:34:24,077: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-09 18:34:24,078: INFO: 1131424634: File already exists of size: ~ 23 KB]
